## Sentiment Analysis using Recurrent Neural Network

### Make sure torch module exists in the workspace

In [3]:
!conda update conda 


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36
  package name: conda




In [2]:
!conda install pytorch -c pytorch -y

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.12
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/amazonei_mxnet_p36

  added / updated specs: 
    - pytorch


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.6.16          |           py36_1         156 KB

The following NEW packages will be INSTALLED:

    cudatoolkit: 10.0.130-0                                   
    ninja:       1.9.0-py36hfd86e86_0                         
    pytorch:     1.1.0-py3.6_cuda10.0.130_cudnn7.5.1_0 pytorch

The following packages will be UPDATED:

    certifi:     2019.6.16-py36_0                              --> 2019.6.16-py36_1


certifi-2019.6.16    | 156 KB    | ##################################### | 100% 

In [3]:
import torch

### Step 1: Downloading the data

In [1]:

%mkdir ../data
!wget -O ../data/aclImdb_v1.tar.gz http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -zxf ../data/aclImdb_v1.tar.gz -C ../data

--2019-08-06 02:06:38--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘../data/aclImdb_v1.tar.gz’

../data/aclImdb_v1. 100%[===================>]  80.23M  23.6MB/s    in 4.5s    

2019-08-06 02:06:43 (17.7 MB/s) - ‘../data/aclImdb_v1.tar.gz’ saved [84125825/84125825]



## Preparing and Processing Data

### Step 2: Preparing and Processing the data

In [4]:
import os
import glob

def read_imdb_data(data_dir='../data/aclImdb'):
    data = {}
    labels = {}
    
    for data_type in ['train', 'test']:
        data[data_type] = {}
        labels[data_type] = {}
        
        for sentiment in ['pos', 'neg']:
            data[data_type][sentiment] = []
            labels[data_type][sentiment] = []
            
            path = os.path.join(data_dir, data_type, sentiment, '*.txt')
            files = glob.glob(path)
            
            for f in files:
                with open(f) as review:
                    data[data_type][sentiment].append(review.read())
                    # Here we represent a positive review by '1' and a negative review by '0'
                    labels[data_type][sentiment].append(1 if sentiment == 'pos' else 0)
                    
            assert len(data[data_type][sentiment]) == len(labels[data_type][sentiment]), \
                    "{}/{} data size does not match labels size".format(data_type, sentiment)
                
    return data, labels

In [5]:
data, labels = read_imdb_data()
print("IMDB reviews: train = {} pos / {} neg, test = {} pos / {} neg".format(
            len(data['train']['pos']), len(data['train']['neg']),
            len(data['test']['pos']), len(data['test']['neg'])))

IMDB reviews: train = 12500 pos / 12500 neg, test = 12500 pos / 12500 neg


In [6]:
from sklearn.utils import shuffle

def prepare_imdb_data(data, labels):
    """Prepare training and test sets from IMDb movie reviews."""
    
    #Combine positive and negative reviews and labels
    data_train = data['train']['pos'] + data['train']['neg']
    data_test = data['test']['pos'] + data['test']['neg']
    labels_train = labels['train']['pos'] + labels['train']['neg']
    labels_test = labels['test']['pos'] + labels['test']['neg']
    
    #Shuffle reviews and corresponding labels within training and test sets
    data_train, labels_train = shuffle(data_train, labels_train)
    data_test, labels_test = shuffle(data_test, labels_test)
    
    # Return a unified training data, test data, training labels, test labets
    return data_train, data_test, labels_train, labels_test

In [7]:
train_X, test_X, train_y, test_y = prepare_imdb_data(data, labels)
print("IMDb reviews (combined): train = {}, test = {}".format(len(train_X), len(test_X)))

IMDb reviews (combined): train = 25000, test = 25000


In [8]:
print(train_X[100])
print(train_y[100])

mahatma Gandhi, the father of the nation in his quest for India's freedom struggle ignored his own family and son, this movie is about his son Hiralal who feels neglected because of mahatma Gandhi's service to the society. The movie starts off in South Africa where Mahatma Gandhi works as a barrister and fighting the cause of India's freedom against the British. Hirarala arrives in South Africa to help his dad who is a barrister, since gandhi was involved in the freedom struggle, he wanted his wife and children to join in too as a service to the society and as a result hirarlal does not get a chance to complete his education and fails his exams, he gets married to his love gulab (bhoomika Chawla) against his father's wishes. Hiralal has ambitions to travel to england and become a barrister just like his dad but his own dad refuses to grant him a scholarship offered to his family by a businessman and instead gives it away to another person saying that the scholarship should not be limit

In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *

import re
from bs4 import BeautifulSoup

def review_to_words(review):
    nltk.download("stopwords", quiet=True)
    stemmer = PorterStemmer()
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    words = [w for w in words if w not in stopwords.words("english")] # Remove stopwords
    words = [PorterStemmer().stem(w) for w in words] # stem
    
    return words

### Question: What does review_to_words do?

The review_to_words method defined above uses BeautifulSoup to remove any html tags that appear and uses the nltk package to tokenize the reviews. As a check to ensure we know how everything is working, try applying review_to_words to one of the reviews in the training set.

In [10]:
# TODO: Apply review_to_words to a review (train_X[100] or any other review)
review_to_words(train_X[100])

['mahatma',
 'gandhi',
 'father',
 'nation',
 'quest',
 'india',
 'freedom',
 'struggl',
 'ignor',
 'famili',
 'son',
 'movi',
 'son',
 'hiral',
 'feel',
 'neglect',
 'mahatma',
 'gandhi',
 'servic',
 'societi',
 'movi',
 'start',
 'south',
 'africa',
 'mahatma',
 'gandhi',
 'work',
 'barrist',
 'fight',
 'caus',
 'india',
 'freedom',
 'british',
 'hirarala',
 'arriv',
 'south',
 'africa',
 'help',
 'dad',
 'barrist',
 'sinc',
 'gandhi',
 'involv',
 'freedom',
 'struggl',
 'want',
 'wife',
 'children',
 'join',
 'servic',
 'societi',
 'result',
 'hirarl',
 'get',
 'chanc',
 'complet',
 'educ',
 'fail',
 'exam',
 'get',
 'marri',
 'love',
 'gulab',
 'bhoomika',
 'chawla',
 'father',
 'wish',
 'hiral',
 'ambit',
 'travel',
 'england',
 'becom',
 'barrist',
 'like',
 'dad',
 'dad',
 'refus',
 'grant',
 'scholarship',
 'offer',
 'famili',
 'businessman',
 'instead',
 'give',
 'away',
 'anoth',
 'person',
 'say',
 'scholarship',
 'limit',
 'famili',
 'open',
 'deserv',
 'student',
 'live',


In [11]:
import pickle

cache_dir = os.path.join("../cache", "sentiment_analysis")  # where to store cache files
os.makedirs(cache_dir, exist_ok=True)  # ensure cache directory exists

def preprocess_data(data_train, data_test, labels_train, labels_test,
                    cache_dir=cache_dir, cache_file="preprocessed_data.pkl"):
    """Convert each review to words; read from cache if available."""

    # If cache_file is not None, try to read from it first
    cache_data = None
    print(cache_file)
    if cache_file is not None:
        try:
            with open(os.path.join(cache_dir, cache_file), "rb") as f:
                cache_data = pickle.load(f)
            print("Read preprocessed data from cache file:", cache_file)
        except:
            print("File not found")
            pass  # unable to read from cache, but that's okay
    
    # If cache is missing, then do the heavy lifting
    if cache_data is None:
        print('Preprocess training and test data to obtain words for each review')
        # Preprocess training and test data to obtain words for each review
        #words_train = list(map(review_to_words, data_train))
        #words_test = list(map(review_to_words, data_test))
        words_train = [review_to_words(review) for review in data_train]
        words_test = [review_to_words(review) for review in data_test]
        
        # Write to cache file for future runs
        print('Preprocess training and test data to obtain words for each review')
        if cache_file is not None:
            cache_data = dict(words_train=words_train, words_test=words_test,
                              labels_train=labels_train, labels_test=labels_test)
            with open(os.path.join(cache_dir, cache_file), "wb") as f:
                pickle.dump(cache_data, f)
            print("Wrote preprocessed data to cache file:", cache_file)
    else:
        print('Unpack data loaded from cache file')
        # Unpack data loaded from cache file
        words_train, words_test, labels_train, labels_test = (cache_data['words_train'],
                cache_data['words_test'], cache_data['labels_train'], cache_data['labels_test'])
    
    return words_train, words_test, labels_train, labels_test

In [12]:
# Preprocess data
train_X, test_X, train_y, test_y = preprocess_data(train_X, test_X, train_y, test_y)

preprocessed_data.pkl
Read preprocessed data from cache file: preprocessed_data.pkl
Unpack data loaded from cache file


In [13]:
import numpy as np
import re
from collections import Counter

def build_dict(data, vocab_size = 5000):
    """Construct and return a dictionary mapping each of the most frequently appearing words to a unique integer."""
    
    # TODO: Determine how often each word appears in `data`. Note that `data` is a list of sentences and that a
    #       sentence is a list of words.
    # print(np.concatenate( data[0:5], axis=0 ))
    word_counts = Counter(np.concatenate( data, axis=0 ))
    # print(word_counts)
    # word_count = {} # A dict storing the words that appear in the reviews along with how often they occur
    
    # TODO: Sort the words found in `data` so that sorted_words[0] is the most frequently appearing word and
    #       sorted_words[-1] is the least frequently appearing word.
    
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    # print(sorted_words[-1])
    
    word_dict = {} # This is what we are building, a dictionary that translates words into integers
    for idx, word in enumerate(sorted_words[:vocab_size - 2]): # The -2 is so that we save room for the 'no word'
        word_dict[word] = idx + 2                              # 'infrequent' labels
        
    return word_dict

In [14]:
word_dict = build_dict(train_X)
print(word_dict)

{'movi': 2, 'film': 3, 'one': 4, 'like': 5, 'time': 6, 'good': 7, 'make': 8, 'charact': 9, 'get': 10, 'see': 11, 'watch': 12, 'stori': 13, 'even': 14, 'would': 15, 'realli': 16, 'well': 17, 'scene': 18, 'look': 19, 'show': 20, 'much': 21, 'end': 22, 'peopl': 23, 'bad': 24, 'go': 25, 'great': 26, 'also': 27, 'first': 28, 'love': 29, 'think': 30, 'way': 31, 'act': 32, 'play': 33, 'made': 34, 'thing': 35, 'could': 36, 'know': 37, 'say': 38, 'seem': 39, 'work': 40, 'plot': 41, 'two': 42, 'actor': 43, 'year': 44, 'come': 45, 'mani': 46, 'seen': 47, 'take': 48, 'want': 49, 'life': 50, 'never': 51, 'littl': 52, 'best': 53, 'tri': 54, 'man': 55, 'ever': 56, 'give': 57, 'better': 58, 'still': 59, 'perform': 60, 'find': 61, 'feel': 62, 'part': 63, 'back': 64, 'use': 65, 'someth': 66, 'director': 67, 'actual': 68, 'interest': 69, 'lot': 70, 'real': 71, 'old': 72, 'cast': 73, 'though': 74, 'live': 75, 'star': 76, 'enjoy': 77, 'guy': 78, 'anoth': 79, 'new': 80, 'role': 81, 'noth': 82, '10': 83, 'fu

### Question: What are the five most frequently appearing words?

<b>Answer:</b> 'movi', 'film', 'one', 'like', 'time'

"movi" word is nearly 51695 times

it doesn't make sence in training because 'movi', 'film', 'one', 'time' has <b>no sentiment.</b>

In [16]:
# TODO: Use this space to determine the five most frequently appearing words in the training set.
word_counts = Counter(np.concatenate( train_X, axis=0 ))
sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
print(sorted_words[0:5])
print(word_counts['movi'])

['movi', 'film', 'one', 'like', 'time']
51695


In [17]:
data_dir = '../data/pytorch' # The folder we will use for storing data
if not os.path.exists(data_dir): # Make sure that the folder exists
    os.makedirs(data_dir)

In [18]:
with open(os.path.join(data_dir, 'word_dict.pkl'), "wb") as f:
    pickle.dump(word_dict, f)

In [19]:
def convert_and_pad(word_dict, sentence, pad=500):
    NOWORD = 0 # We will use 0 to represent the 'no word' category
    INFREQ = 1 # and we use 1 to represent the infrequent words, i.e., words not appearing in word_dict
    
    working_sentence = [NOWORD] * pad
    
    for word_index, word in enumerate(sentence[:pad]):
        if word in word_dict:
            working_sentence[word_index] = word_dict[word]
        else:
            working_sentence[word_index] = INFREQ
            
    return working_sentence, min(len(sentence), pad)

def convert_and_pad_data(word_dict, data, pad=500):
    result = []
    lengths = []
    
    for sentence in data:
        converted, leng = convert_and_pad(word_dict, sentence, pad)
        result.append(converted)
        lengths.append(leng)
        
    return np.array(result), np.array(lengths)

In [20]:
train_X, train_X_len = convert_and_pad_data(word_dict, train_X)
test_X, test_X_len = convert_and_pad_data(word_dict, test_X)

In [21]:
# Use this cell to examine one of the processed reviews to make sure everything is working as intended.
print(len(train_X[0]))
print(train_X_len[0:1]) # 500 - 333 (non zeros) = 167

500
[42]


### Question: Understanding preprocess_data and convert_and_pad_data

Answer:

* <b>preprocess_data</b> function will cut the data preparation cost, since it is one time process and the result are saved in a file. In case if the notebook instance terminal connection is broken then we need to restart from begining in that case we can skip the data preparation step and proceed to training by directly loading the preprocessed data from file
* <b>convert_and_pad_data</b> this function helps to maintain a constant data length. since we using LSTM Clasifier with Embedding so we need to maintain a constant batch size throughout the training

## Build and Train the PyTorch Model

### Step 3: Upload the data to S3

In [22]:
import pandas as pd
    
pd.concat([pd.DataFrame(train_y), pd.DataFrame(train_X_len), pd.DataFrame(train_X)], axis=1) \
        .to_csv(os.path.join(data_dir, 'train.csv'), header=False, index=False)

In [23]:
import sagemaker

sagemaker_session = sagemaker.Session()

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/sentiment_rnn'

role = sagemaker.get_execution_role()

In [24]:
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

## Writing the training method

### Step 4: Build and Train the PyTorch Model

In [25]:
import torch

In [26]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    """
    This is the simple RNN model we will be using to perform Sentiment Analysis.
    """

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        """
        Initialize the model by settingg up the various layers.
        """
        super(LSTMClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.dense = nn.Linear(in_features=hidden_dim, out_features=1)
        self.sig = nn.Sigmoid()
        
        self.word_dict = None

    def forward(self, x):
        """
        Perform a forward pass of our model on some input.
        """
        x = x.t()
        lengths = x[0,:]
        reviews = x[1:,:]
        embeds = self.embedding(reviews)
        lstm_out, _ = self.lstm(embeds)
        out = self.dense(lstm_out)
        out = out[lengths - 1, range(len(lengths))]
        return self.sig(out.squeeze())

In [27]:
import torch
import torch.utils.data
import pandas as pd

# Read in only the first 250 rows
train_sample = pd.read_csv(os.path.join(data_dir, 'train.csv'), header=None, names=None, nrows=250)

# Turn the input pandas dataframe into tensors
train_sample_y = torch.from_numpy(train_sample[[0]].values).float().squeeze()
train_sample_X = torch.from_numpy(train_sample.drop([0], axis=1).values).long()

# Build the dataset
train_sample_ds = torch.utils.data.TensorDataset(train_sample_X, train_sample_y)
# Build the dataloader
train_sample_dl = torch.utils.data.DataLoader(train_sample_ds, batch_size=50)

## Training the model

In [31]:
def train(model, train_loader, epochs, optimizer, loss_fn, device):
    for epoch in range(1, epochs + 1):
        model.train()
        total_loss = 0
        for batch in train_loader:         
            batch_X, batch_y = batch
            
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # model.zero_grad()
            optimizer.zero_grad()
            
            # TODO: Complete this train method to train the model provided.
            output = model(batch_X)
            loss = loss_fn(output, batch_y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data.item()
        print("Epoch: {}, BCELoss: {}".format(epoch, total_loss / len(train_loader)))

In [32]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [35]:
import torch.optim as optim
#from train.model import LSTMClassifier

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(32, 200, 5000).to(device)
optimizer = optim.Adam(model.parameters())
loss_fn = torch.nn.BCELoss()

train(model, train_sample_dl, 10, optimizer, loss_fn, device)

Epoch: 1, BCELoss: 0.6929407000541687
Epoch: 2, BCELoss: 0.6789801836013794
Epoch: 3, BCELoss: 0.6659158825874328
Epoch: 4, BCELoss: 0.6480816960334778
Epoch: 5, BCELoss: 0.6173786520957947
Epoch: 6, BCELoss: 0.5791762709617615
Epoch: 7, BCELoss: 0.5133359491825104
Epoch: 8, BCELoss: 0.4658102333545685
Epoch: 9, BCELoss: 0.39789999127388
Epoch: 10, BCELoss: 0.3329601466655731


In [44]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(entry_point="train.py",
                    source_dir="train",
                    role=role,
                    framework_version='0.4.0',
                    train_instance_count=1,
                    train_instance_type='ml.m4.xlarge',
                    hyperparameters={
                        'epochs': 10,
                        'hidden_dim': 200,
                    })

In [45]:
# this function loads the already completed training job to the Estimater
# For first please comment it

# my_training_job_name = 'sagemaker-pytorch-2019-02-19-16-44-24-823'
# estimator = PyTorch.attach(my_training_job_name)

In [ ]:
estimator.fit({'training': input_data})

2019-08-06 19:28:05 Starting - Starting the training job...
2019-08-06 19:28:06 Starting - Launching requested ML instances......
2019-08-06 19:29:08 Starting - Preparing the instances for training...
2019-08-06 19:30:05 Downloading - Downloading input data...
2019-08-06 19:30:30 Training - Downloading the training image...
2019-08-06 19:30:48 Training - Training image download completed. Training in progress.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2019-08-06 19:30:49,209 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2019-08-06 19:30:49,212 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2019-08-06 19:30:49,225 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2019-08-06 19:30:52,252 sagemaker_pytorch_container.training INFO     Invoking user training script.
2019-08-06 19:30:52,477 sagemaker-containers INFO 

Epoch: 1, BCELoss: 0.6740777468194767
Epoch: 2, BCELoss: 0.631865820106195
Epoch: 3, BCELoss: 0.5317807757124609
Epoch: 4, BCELoss: 0.4539955386093685
Epoch: 5, BCELoss: 0.3916754898976307
Epoch: 6, BCELoss: 0.3532942174648752
Epoch: 7, BCELoss: 0.3393844743164218
Epoch: 8, BCELoss: 0.3043571880885533
Epoch: 9, BCELoss: 0.3032935967250746
Epoch: 10, BCELoss: 0.28959563617803613
2019-08-06 21:15:43,071 sagemaker-containers INFO     Reporting training SUCCESS

2019-08-06 21:15:51 Uploading - Uploading generated training model
2019-08-06 21:15:51 Completed - Training job completed
Billable seconds: 6346


### Step 5: Testing the model

### Step 6: Deploy the model for testing


In [47]:
# TODO: Deploy the trained model
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')


---------------------------------------------------------------------------------------!

In [48]:
#sagemaker-pytorch-2019-08-06-19-28-05-265
# this function loads the already completed training job to the Estimater
# For first please comment it

# my_training_job_name = 'sagemaker-pytorch-2019-08-06-19-28-05-265'
# estimator = PyTorch.attach(my_training_job_name)

## Writing inference code

### Step 7 - Use the model for testing

In [49]:
test_X = pd.concat([pd.DataFrame(test_X_len), pd.DataFrame(test_X)], axis=1)

In [50]:
# We split the data into chunks and send each chunk seperately, accumulating the results.

def predict(data, rows=512):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = np.array([])
    for array in split_array:
        predictions = np.append(predictions, predictor.predict(array))
    
    return predictions

In [51]:
predictions = predict(test_X.values)
predictions = [round(num) for num in predictions]

In [52]:
from sklearn.metrics import accuracy_score
accuracy_score(test_y, predictions)

0.85292

### Question: How does this model compare to the XGBoost model?

<b>Answer:</b> XGBoost is an gradient boosting library designed to handle smaller variable problems with lesser training and more accuracy. But Deep learning is designed to large variable data supervised/un-supervised learning but it need more training For sentiment analysis XGBoost is better because XGBoost is the state of the art in most regression and classification problems with better result and less training. since it is mono label classification problem

## (TODO) More testing


In [53]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'

In [54]:
data_X = None
data_len = None
input_data_words = review_to_words(test_review)
data_X, data_len = convert_and_pad_data(word_dict, input_data_words)
# print(data_len)
# print(data_X[0])
data_pack = np.column_stack((data_len, data_X))
print(data_pack)
data_pack = data_pack.reshape(1, -1)

data = torch.from_numpy(data_pack)
data = data.to(device)

[[   8    1    1 ...    0    0    0]
 [   7 1440 1222 ...    0    0    0]
 [   4 1222    1 ...    0    0    0]
 ...
 [   3 1807    1 ...    0    0    0]
 [   7 1097 1496 ...    0    0    0]
 [   5  894 1767 ...    0    0    0]]


In [55]:
# TODO: Convert test_review into a form usable by the model and save the results in test_data
test_data = None
predictions = None
test_review_words = review_to_words(test_review)
test_data, test_data_len = convert_and_pad_data(word_dict, test_review_words)
test_data = pd.concat([pd.DataFrame(test_data_len), pd.DataFrame(test_data)], axis=1)
print(test_data)

    0     0     1     2     3     4     5     6     7    8    ...  490  491  \
0     8     1     1     1  1440  1222   704     1     1    0  ...    0    0   
1     7  1440  1222   704     1     1  1097  1206     0    0  ...    0    0   
2     4  1222     1  1046   704     0     0     0     0    0  ...    0    0   
3     4   425   704     1     1     0     0     0     0    0  ...    0    0   
4     4  1046     1  1222     1     0     0     0     0    0  ...    0    0   
5     3     1  1496   704     0     0     0     0     0    0  ...    0    0   
6     6   894     1     1   425     1  1496     0     0    0  ...    0    0   
7     6  1206     1     1  1767   704  1206     0     0    0  ...    0    0   
8     5   425     1     1     1   894     0     0     0    0  ...    0    0   
9     8     1     1     1  1206     1  1222     1  1496    0  ...    0    0   
10    4  1222     1  1730   704     0     0     0     0    0  ...    0    0   
11    8     1     1  1730   704  1496     1  1097  1

In [56]:
predictor.predict(test_data.values)

array([0.48007214, 0.3994868 , 0.2166497 , 0.54820126, 0.42972612,
       0.3945137 , 0.6157694 , 0.32787687, 0.53193456, 0.58629394,
       0.36237544, 0.5513351 , 0.5443086 , 0.6112492 , 0.6231468 ,
       0.47983134, 0.32519004, 0.52465135, 0.67716765, 0.5257694 ],
      dtype=float32)

In [ ]:
estimator.delete_endpoint()

## Deploy the model for the web app

In [ ]:
from sagemaker.predictor import RealTimePredictor
from sagemaker.pytorch import PyTorchModel

class StringPredictor(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(StringPredictor, self).__init__(endpoint_name, sagemaker_session, content_type='text/plain')

model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                     framework_version='0.4.0',
                     entry_point='predict.py',
                     source_dir='serve',
                     predictor_cls=StringPredictor)
predictor = model.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

## Testing the model

In [ ]:
import glob

def test_reviews(data_dir='../data/aclImdb', stop=250):
    
    results = []
    ground = []
    
    # We make sure to test both positive and negative reviews    
    for sentiment in ['pos', 'neg']:
        
        path = os.path.join(data_dir, 'test', sentiment, '*.txt')
        files = glob.glob(path)
        
        files_read = 0
        
        print('Starting ', sentiment, ' files')
        
        # Iterate through the files and send them to the predictor
        for f in files:
            with open(f) as review:
                # First, we store the ground truth (was the review positive or negative)
                if sentiment == 'pos':
                    ground.append(1)
                else:
                    ground.append(0)
                # Read in the review and convert to 'utf-8' for transmission via HTTP
                review_input = review.read().encode('utf-8')
                # Send the review to the predictor and store the results
                result_s = predictor.predict(review_input).decode('UTF-8')
                result_s = int(float(result_s))
                # review_input = int.from_bytes(review_input, byteorder='big', signed=True)
                # print(result_s)
                results.append(result_s)
                
            # Sending reviews to our endpoint one at a time takes a while so we
            # only send a small number of reviews
            files_read += 1
            if files_read == stop:
                break
            
    return ground, results

In [ ]:
ground, results = test_reviews()

In [ ]:
out = torch.tensor(0.55)
result = np.round(out.item(),0)

print(result)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(ground, results)

In [ ]:
test_review = 'The simplest pleasures in life are the best, and this film is one of them. Combining a rather basic storyline of love and adventure this movie transcends the usual weekend fair with wit and unmitigated charm.'
predictor.predict(test_review).decode('UTF-8')

## Setting up Lambda

In [ ]:
# We need to use the low-level library to interact with SageMaker since the SageMaker API
# is not available natively through Lambda.
import json
import boto3

def lambda_handler(event, context):

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = 'sagemaker-pytorch-2019-02-20-17-49-05-967',    # The name of the endpoint we created
                                       ContentType = 'text/plain',                 # The data format that is expected
                                       Body = event['body'])                       # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')
    result = int(float(result))

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : result
    }

In [ ]:
predictor.endpoint

## Setting up API Gateway
### Deploying our web app

In [ ]:
predictor.delete_endpoint()